In [1]:
import sqlite3
import re
import inspect
import collections
import importlib
import xml.etree.ElementTree as ET

import Android as android
# from Android.IntentFilter import IntentFilter
from Android.Os.Bundle import Bundle
from Android.Os.Parcel import Parcel
from Android.content.res.Resources import Resources
from Android.content.pm.ActivityInfo import ActivityInfo
from Android.content.pm.ApplicationInfo import ApplicationInfo
from Android.content.pm.PackageInfo  import PackageInfo
from Android.content.pm.ProviderInfo import ProviderInfo
import TestActivity 


In [2]:
NS_MAP = "xmlns:map"

def parse_nsmap(file):
    events = "start", "start-ns", "end-ns"
    root = None
    ns_map = []
    for event, elem in ET.iterparse(file, events):
        if event == "start-ns":
            ns_map.append(elem)
        elif event == "end-ns":
            ns_map.pop()
        elif event == "start":
            if root is None:
                root = elem
            elem.set(NS_MAP, dict(ns_map))
    return ET.ElementTree(root)


In [3]:
class AndroidManifest(android.Object):
    def __init__(self, filepath, context):
        self._filepath = filepath
        self._resources = Resources(context)

    @staticmethod
    def get_manifest_attrib_name(tag_name):
        equiv = {
            'compatible-screens':'compatible-screens-screen',
            'manifest':'',
            'uses-permission-sdk-23':'uses-permission'
                }
        tag_name = equiv.get(tag_name, tag_name)
        attrname = 'AndroidManifest' + tag_name.title().replace('-', '')
        assert hasattr(android.R.styleable, attrname), 'No tag description'
        return attrname
    
    def get_manifest_tag_attribs(self, attrname):
        res = self._resources
        styleable_id = getattr(android.R.styleable, attrname)
        styleable_id = filter(lambda x: bool(x), styleable_id)
        array = map(lambda x:res._unpack_pointer(x).attrib, styleable_id)
        return [(x['name'], x.get('format', 'String')) for x in array]
        
    @staticmethod
    def clasifyFields(fields, aninst):
        def fromFlagToField(x):
            w = x.split('_')[1:]
            return reduce(lambda x, y: x + y.title(), w[1:], w[0].lower())
        flagnames = filter(lambda x: x.startswith('FLAG_'), dir(inspect.getmodule(aninst)))
        flags = map(fromFlagToField, flagnames)
        fieldsFlags = set(fields).intersection(flags)
        attribs = sorted(vars(aninst).keys())
        fieldsAttribs = set(fields).intersection(attribs)
        unclassFields = set(fields).difference(fieldsFlags.union(fieldsAttribs))
        return fieldsFlags, fieldsAttribs, unclassFields
    
    def parse_component(self, etelement):
        try:
            etelement.attrib.pop('xmlns:map')
        except:
            pass
        # Se limpian los prefijos de los items
        attribitems = etelement.attrib.items()
        keys, values = zip(*attribitems)
        keys = [x.split('}')[-1] for x in keys]
        attribset = dict(zip(keys, values))
        
        manifest_attr_name = self.get_manifest_attrib_name(etelement.tag)
        styleable_id = getattr(android.R.styleable, manifest_attr_name)
        fieldvalues = self._resources.obtainAtributes(attribset, styleable_id)
        fieldnames, fieldtypes = zip(*self.get_manifest_tag_attribs(manifest_attr_name))
        answ = [x for x in zip(fieldnames,fieldvalues) if x[0] in attribset]
        attribset.update(answ)
        return attribset.items()
    
    @staticmethod
    def parse_nsmap(file):
        NS_MAP = "xmlns:map"
        events = "start", "start-ns", "end-ns"
        root = None
        ns_map = []
        for event, elem in ET.iterparse(file, events):
            if event == "start-ns":
                ns_map.append(elem)
            elif event == "end-ns":
                ns_map.pop()
            elif event == "start":
                if root is None:
                    root = elem
                elem.set(NS_MAP, dict(ns_map))
        return ET.ElementTree(root)
    
    def processAndroidManifest(self, manifest_path):
        manifest_file = self._filepath
#         db = self.getWritableDatabase()
#         insertValueMap = lambda table_class, valueMap: \
#             db.insert(table_class.TABLE_NAME, None, valueMap)
        db = []
        insertValueMap = lambda table_class, valueMap: \
            db.append((table_class, None, valueMap))
    

        stack = []
        componentStack = []

        manifest = self.parse_nsmap(manifest_file).getroot()
        package = manifest.attrib['package']

        valueMap = dict(name=package, path=manifest_path)
        insertValueMap('PACKAGE_TABLE', valueMap)
        package_id = len(db)

        stack.append((-1, manifest))
        while stack:
            parent_id, element = stack.pop()
            if parent_id != -1 and componentStack[-1][0] != parent_id:
                component_id, componentinfo = componentStack.pop()
                print 'update db component info', component_id, componentinfo
                parcel = Parcel()
                componentinfo.writeToParcel(parcel, 0)
                content = parcel.marshall()
                table_class, dummy, valueMap = db[component_id]
                valueMap['content'] = content
                db[component_id] = (table_class, dummy, valueMap)
            # items = [(key.split('}')[-1], value) for key, value in element.items()]
            component = self.componentFactory(element.tag)
            if component:
                if element.tag == 'intent-filter':
                    component.readFromXml(element)
                else:
                    elementparse = self.parse_component(element)
                    flags = self.setFlagsValue(elementparse, component)
                    attrs = self.setAttribsValue(elementparse, component)
                    ufields = self.setUnclassifiedFields(elementparse, component, flags + attrs)
                valueMap = dict(package_id=package_id, parent=parent_id,
                                tag_type=element.tag, content='')
                insertValueMap('COMPONENTS_TABLE', valueMap)
                tagid = len(db)
                componentStack.append((tagid, component))
                stack.extend([(tagid, item) for item in element])
        return db
    
    @staticmethod
    def setFlagsValue(itemparse, aninst):
        def fromFlagToField(x):
            w = x.split('_')[1:]
            return reduce(lambda x, y: x + y.title(), w[1:], w[0].lower())
        itemmap = dict(itemparse)
        module = inspect.getmodule(aninst)
        flagnames = filter(lambda x: x.startswith('FLAG_'), dir(module))
        if flagnames: 
            flagnames = filter(lambda x: itemmap.get(fromFlagToField(x), False), flagnames)
            flags = reduce(lambda x, y: x | getattr(module, y), flagnames, 0)
            aninst.flags = flags
        return map(fromFlagToField, flagnames)

    @staticmethod
    def setAttribsValue(itemparse, aninst):
        itemmap = dict(itemparse)
        attribs = sorted(vars(aninst).keys())
        fieldsAttribs = set(attribs).intersection(itemmap)
        map(lambda x: setattr(aninst, x, itemmap[x]), fieldsAttribs)
        return list(fieldsAttribs)
    
    @staticmethod
    def setUnclassifiedFields(itemparse, aninst, classifiedFields):
        bundle = Bundle()
        itemmap = {key:value for key, value in itemparse if key not in classifiedFields}
        map(lambda x: bundle.putString(x[0], '%s' % x[1]), itemmap.items())
        aninst._unclassifiedFields = bundle
        return itemmap.keys()
    
    @staticmethod
    def componentFactory(elemtag):
        equiv = {
            'manifest': '.pm.PackageInfo',
            'application': '.pm.ApplicationInfo',
            'activity': '.pm.ActivityInfo',
            'provider': '.pm.ProviderInfo',
            'intent-filter': '.IntentFilter'
        }
        classname = equiv.get(elemtag, None)
        if classname:
            modulename = importlib.import_module(classname, 'Android.content')
            cls = getattr(modulename, classname.rsplit('.', 1)[-1])
            return cls()
    

In [4]:
manifest_path = '/media/amontesb/HITACHI/AndroidApps/TestActivity'
manifest_file = '/media/amontesb/HITACHI/AndroidApps/TestActivity/AndroidManifest.xml'
context = dict(android=android.R)
manifestroot = parse_nsmap(manifest_file)

In [5]:
manifest = manifestroot.getroot()
application = manifest[0]
provider = application[0]
activity = application[1]  # ativities = application[1:3]

In [6]:
am = AndroidManifest(manifest_file, context)
self = am

In [7]:
        db = []
        insertValueMap = lambda table_class, valueMap: \
            db.append((table_class, None, valueMap))
    

        stack = []
        componentStack = []

        manifest = self.parse_nsmap(manifest_file).getroot()
        package = manifest.attrib['package']

        valueMap = dict(name=package, path=manifest_path)
        insertValueMap('PACKAGE_TABLE', valueMap)
        package_id = len(db)

        stack.append((-1, manifest))

In [8]:
package, package_id, stack

('com.AdroidApps.TestActivity',
 1,
 [(-1, <Element 'manifest' at 0x7fa9b43b55d0>)])

In [9]:
componentStack[-1][0], stack[-1][0]

IndexError: list index out of range

In [36]:
componentStack[-1][1].activityInfo is None

AttributeError: 'ActivityInfo' object has no attribute 'activityInfo'

In [37]:
            parent_id, element = stack.pop()
            if parent_id != -1 and componentStack[-1][0] != parent_id:
                component_id, componentinfo = componentStack.pop()
                print 'update db component info', component_id, componentinfo
                parcel = Parcel()
                componentinfo.writeToParcel(parcel, 0)
                content = parcel.marshall()
                table_class, dummy, valueMap = db[component_id]
                valueMap['content'] = content
                db[component_id] = (table_class, dummy, valueMap)

update db component info 4 <Android.content.pm.ActivityInfo.ActivityInfo object at 0x7fa9b43cf0e0>


error: 'i' format requires -2147483648 <= number <= 2147483647

In [26]:
element

<Element 'activity' at 0x7fa9b43b5890>

In [27]:
            # items = [(key.split('}')[-1], value) for key, value in element.items()]
            component = self.componentFactory(element.tag)

In [28]:
component

In [29]:
            if component:
                if element.tag == 'intent-filter':
                    component.readFromXml(element)
                else:
                    elementparse = self.parse_component(element)
                    flags = self.setFlagsValue(elementparse, component)
                    attrs = self.setAttribsValue(elementparse, component)
                    ufields = self.setUnclassifiedFields(elementparse, component, flags + attrs)
                valueMap = dict(package_id=package_id, parent=parent_id,
                                tag_type=element.tag, content='')
                insertValueMap('COMPONENTS_TABLE', valueMap)
                tagid = len(db)
                componentStack.append((tagid, component))
                stack.extend([(tagid, item) for item in element])

In [30]:
stack, componentStack

([(3, <Element 'provider' at 0x7fa9b43b5650>),
  (3, <Element 'activity' at 0x7fa9b43b5750>)],
 [(2, <Android.content.pm.PackageInfo.PackageInfo at 0x7fa9b4110f80>),
  (3, <Android.content.pm.ApplicationInfo.ApplicationInfo at 0x7fa9b4113170>),
  (4, <Android.content.pm.ActivityInfo.ActivityInfo at 0x7fa9b43cf0e0>)])

In [31]:
db

[('PACKAGE_TABLE',
  None,
  {'name': 'com.AdroidApps.TestActivity',
   'path': '/media/amontesb/HITACHI/AndroidApps/TestActivity'}),
 ('COMPONENTS_TABLE',
  None,
  {'content': '', 'package_id': 1, 'parent': -1, 'tag_type': 'manifest'}),
 ('COMPONENTS_TABLE',
  None,
  {'content': '', 'package_id': 1, 'parent': 2, 'tag_type': 'application'}),
 ('COMPONENTS_TABLE',
  None,
  {'content': '', 'package_id': 1, 'parent': 3, 'tag_type': 'activity'})]

In [ ]:
vars(aninst)

In [ ]:
styleablename = am.get_manifest_attrib_name('Application')
styleattribs = am.get_manifest_tag_attribs(styleablename)
sorted(styleattribs)

In [ ]:
itemparse = am.parse_component(application)
itemparse

In [ ]:
fields, fieldtypes = zip(*styleattribs)
fields

In [ ]:
android.R.styleable.AndroidManifest_versionName

In [ ]:
aninst = ApplicationInfo()
aninst

In [ ]:
am.clasifyFields(fields, aninst)

In [ ]:
styleable_id = android.R.styleable.AndroidManifest
array = map(lambda x:am._resources._unpack_pointer(x).attrib, styleable_id)
[(x['name'], x.get('format', 'String')) for x in array]

In [ ]:
rid = android.R.attr.installLocation

In [ ]:
elem = am._resources._unpack_pointer(rid)
print elem.attrib
for item in elem:
    print item.attrib

In [ ]:
context = dict(android=android.R)
res = Resources(context)

In [ ]:
def unpack_attribs(styleable_id):
    styleable_id = filter(lambda x: bool(x), styleable_id)
    array = map(lambda x:res._unpack_pointer(x).attrib, styleable_id)
    return [(x['name'], x.get('format', 'String')) for x in array]

#### Tipo de componente y resid

In [ ]:
def get_manifest_attrib_name(tag_name):
    equiv = {
        'compatible-screens':'compatible-screens-screen',
        'manifest':'',
        'uses-permission-sdk-23':'uses-permission'
            }
    tag_name = equiv.get(tag_name, tag_name)
    attrname = 'AndroidManifest' + tag_name.title().replace('-', '')
    assert hasattr(android.R.styleable, attrname), 'No tag description'
    return attrname

In [ ]:
def get_manifest_tag_attibs(tag_name):
    attrname = get_manifest_attrib_name(tag_name)
    styleable_id = getattr(android.R.styleable, attrname)
    styleable_id = filter(lambda x: bool(x), styleable_id)
    array = map(lambda x:res._unpack_pointer(x).attrib, styleable_id)
    return [(x['name'], x.get('format', 'String')) for x in array]

In [ ]:
manifest_tags = [
    '<action>',
     '<activity>',
     '<activity-alias>',
     '<application>',
     '<category>',
     '<compatible-screens>', # <compatible-screens-screen>
     '<data>',
     '<grant-uri-permission>',
     '<instrumentation>',
     '<intent-filter>',
     '<manifest>', # <>
     '<meta-data>',
     '<path-permission>',
     '<permission>',
     '<permission-group>',
     '<permission-tree>',
     '<provider>',
     '<receiver>',
     '<service>',
     '<supports-gl-texture>',
     '<supports-screens>',
     '<uses-configuration>',
     '<uses-feature>',
     '<uses-library>',
     '<uses-permission>',
     '<uses-permission-sdk-23>', # '<uses-permission>'
     '<uses-sdk>'
]

In [ ]:
get_manifest_tag_attibs('manifest')

In [ ]:
filepath = '/media/amontesb/HITACHI/AndroidApps/TestActivity/AndroidManifest.xml'

In [ ]:
manifest = parse_nsmap(filepath).getroot()
manifest.tag

In [ ]:
application = manifest[0]
application.tag

In [ ]:
for k, elem in enumerate(application):
    print k, elem.tag

In [ ]:
activity = application[1]
activity.tag

In [ ]:
activity.attrib

In [ ]:
provider = application[0]
provider.tag

In [ ]:
def parse_component(etelement):
    try:
        etelement.attrib.pop('xmlns:map')
    except:
        pass
    # Se limpian los prefijos de los items
    attribitems = etelement.attrib.items()
    keys, values = zip(*attribitems)
    keys = [x.split('}')[-1] for x in keys]
    attribset = dict(zip(keys, values))
    manifest_attr_name = get_manifest_attrib_name(etelement.tag)
    styleable_id = getattr(android.R.styleable, manifest_attr_name)
    fieldvalues = res.obtainAtributes(attribset, styleable_id)
    fieldnames = zip(*get_manifest_tag_attibs(etelement.tag))[0]
    answ = [x for x in zip(fieldnames,fieldvalues) if x[0] in attribset]
    attribset.update(answ)
    return attribset.items()

In [ ]:
parse_component(provider)

In [ ]:
parse_component(manifest)

In [ ]:
parse_component(activity)

In [ ]:
filepath = r'/media/amontesb/HITACHI/AndroidApps/Android/_tests/data/testAndroidManifest.xml'

In [ ]:
manifest = parse_nsmap(filepath).getroot()
manifest.tag

In [ ]:
application = manifest[0]
application.tag

In [ ]:
for k, elem in enumerate(application):
    print k, elem.tag

In [ ]:
parse_component(application)

In [ ]:
android.R.string.action_bar_home_description

In [ ]:
res.getValue('@android:string/action_bar_home_description', None)

In [ ]:
android.R.style.Animation__Holo

In [ ]:
android.R.drawable.action_bar_divider

In [ ]:
import inspect
from Android.content.pm.ApplicationInfo import ApplicationInfo
from Android.content.pm.ActivityInfo import ActivityInfo

In [ ]:
def clasifyFields(fields, aninst):
    def fromFlagToField(x):
        w = x.split('_')[1:]
        return reduce(lambda x, y: x + y.title(), w[1:], w[0].lower())
    flagnames = filter(lambda x: x.startswith('FLAG_'), dir(inspect.getmodule(aninst)))
    flags = map(fromFlagToField, flagnames)
    fieldsFlags = set(fields).intersection(flags)
    attribs = sorted(vars(aninst).keys())
    fieldsAttribs = set(fields).intersection(attribs)
    unclassFields = set(fields).difference(fieldsFlags.union(fieldsAttribs))
    return fieldsFlags, fieldsAttribs, unclassFields

In [ ]:
tagname = 'Activity'
tag_attribs = get_manifest_tag_attibs(tagname)

fields, fieldtypes = zip(*tag_attribs)
aninst = ActivityInfo()

In [ ]:
flags, attribs, _fields = clasifyFields(fields, aninst)
flags, attribs, _fields

In [ ]:
aninst.loadLabel

In [ ]:
def processAndroidManifest(self, manifest_path):
        import pickle
        from Android.IntentFilter import IntentFilter

        manifest_file = os.path.join(manifest_path, 'AndroidManifest.xml')
        db = self.getWritableDatabase()
        insertValueMap = lambda table_class, valueMap: \
            db.insert(table_class.TABLE_NAME, None, valueMap)

        stack = collections.deque()
        root = self.parse_nsmap(manifest_file).getroot()
        package = root.attrib.pop('package')

        valueMap = dict(name=package, path=manifest_path)
        package_id = insertValueMap(PACKAGE_TABLE, valueMap)

        stack.append((-1, root))
        while stack:
            parent_id, element = stack.popleft()
            # items = [(key.split('}')[-1], value) for key, value in element.items()]
            tag, attrib = element.tag, element.attrib
            if tag == 'intent-filter':
                ifilter = IntentFilter()
                ifilter.readFromXml(element)
                content = pickle.dumps(ifilter)
                element = []
            else:
                content = ' '.join(['%s="%s"' % x for x in attrib.items()])
            valueMap = dict(package_id=package_id, parent=parent_id,
                            tag_type=tag, content=content)
            tagid = insertValueMap(COMPONENTS_TABLE, valueMap)
            stack.extend([(tagid, item) for item in element])
